# Advanced Molecular Vibration Analysis

This notebook demonstrates advanced workflows for analyzing molecular vibrations:

## Topics Covered

1. Batch processing multiple calculations
2. Comparing vibrational modes across molecules
3. IR spectrum visualization
4. Identifying characteristic functional group vibrations
5. Creating publication-quality figures
6. Exporting visualizations

## Prerequisites

```bash
pip install plotlymol matplotlib pandas
```

## Setup

In [ ]:
# Import required libraries
from plotlymol3d import (
    parse_vibrations,
    draw_3D_rep,
    add_vibrations_to_figure,
    create_vibration_animation,
)
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Dict
import plotly.graph_objects as go

## 1. Batch Processing Multiple Calculations

Process multiple frequency calculations at once for comparative analysis.

In [ ]:
# Define a function to batch process vibration files
def batch_parse_vibrations(file_list: List[str]) -> Dict[str, 'VibrationalData']:
    """
    Parse multiple vibration files at once.
    
    Args:
        file_list: List of file paths to parse
        
    Returns:
        Dictionary mapping filenames to VibrationalData objects
    """
    results = {}
    
    for filepath in file_list:
        try:
            filename = Path(filepath).stem
            vib_data = parse_vibrations(filepath)
            results[filename] = vib_data
            print(f"✓ Parsed {filename}: {len(vib_data.modes)} modes")
        except Exception as e:
            print(f"✗ Failed to parse {filepath}: {e}")
    
    return results

# Example: Process multiple files
file_list = [
    "path/to/molecule1.log",
    "path/to/molecule2.log",
    "path/to/molecule3.log",
]

vib_datasets = batch_parse_vibrations(file_list)

## 2. Creating a Vibrational Mode Summary Table

In [ ]:
def create_mode_summary(vib_data) -> pd.DataFrame:
    """
    Create a pandas DataFrame summarizing all vibrational modes.
    
    Args:
        vib_data: VibrationalData object
        
    Returns:
        DataFrame with mode information
    """
    data = []
    
    for mode in vib_data.modes:
        max_displacement = np.max(vib_data.get_displacement_magnitudes(mode.mode_number))
        avg_displacement = np.mean(vib_data.get_displacement_magnitudes(mode.mode_number))
        
        data.append({
            'Mode': mode.mode_number,
            'Frequency (cm⁻¹)': mode.frequency,
            'IR Intensity (km/mol)': mode.ir_intensity if mode.ir_intensity else np.nan,
            'Imaginary': mode.is_imaginary,
            'Max Displacement (Å)': max_displacement,
            'Avg Displacement (Å)': avg_displacement,
        })
    
    return pd.DataFrame(data)

# Example usage
# Replace 'your_molecule' with an actual key from vib_datasets
if vib_datasets:
    first_molecule = list(vib_datasets.keys())[0]
    summary = create_mode_summary(vib_datasets[first_molecule])
    print(f"\nVibrational mode summary for {first_molecule}:")
    display(summary)

## 3. Simulated IR Spectrum Visualization

Create a bar plot showing IR intensities vs frequencies.

In [ ]:
def plot_ir_spectrum(vib_data, molecule_name: str = "Molecule"):
    """
    Plot a simulated IR spectrum as a bar chart.
    
    Args:
        vib_data: VibrationalData object
        molecule_name: Name for plot title
    """
    # Extract frequencies and intensities
    modes_with_ir = [m for m in vib_data.modes if m.ir_intensity is not None and not m.is_imaginary]
    
    if not modes_with_ir:
        print("No IR intensities available")
        return
    
    frequencies = [m.frequency for m in modes_with_ir]
    intensities = [m.ir_intensity for m in modes_with_ir]
    mode_numbers = [m.mode_number for m in modes_with_ir]
    
    # Create plot
    fig, ax = plt.subplots(figsize=(12, 6))
    
    bars = ax.bar(frequencies, intensities, width=10, color='steelblue', alpha=0.7)
    
    # Add mode numbers as labels for top 5 peaks
    top_5_indices = np.argsort(intensities)[-5:]
    for idx in top_5_indices:
        ax.text(
            frequencies[idx], 
            intensities[idx] + 5,
            f"Mode {mode_numbers[idx]}",
            ha='center',
            fontsize=9,
            rotation=45
        )
    
    ax.set_xlabel('Frequency (cm⁻¹)', fontsize=12)
    ax.set_ylabel('IR Intensity (km/mol)', fontsize=12)
    ax.set_title(f'Simulated IR Spectrum - {molecule_name}', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.set_xlim(min(frequencies) - 100, max(frequencies) + 100)
    
    plt.tight_layout()
    plt.show()
    
    # Print top 5 peaks
    print(f"\nTop 5 IR-active modes for {molecule_name}:")
    print("-" * 60)
    for idx in sorted(top_5_indices, key=lambda i: intensities[i], reverse=True):
        print(f"Mode {mode_numbers[idx]:2d}: {frequencies[idx]:7.1f} cm⁻¹ (IR: {intensities[idx]:6.1f} km/mol)")

# Example usage
if vib_datasets:
    first_molecule = list(vib_datasets.keys())[0]
    plot_ir_spectrum(vib_datasets[first_molecule], first_molecule)

## 4. Interactive IR Spectrum with Plotly

Create an interactive spectrum where you can click on peaks.

In [ ]:
def create_interactive_ir_spectrum(vib_data, molecule_name: str = "Molecule"):
    """
    Create an interactive Plotly IR spectrum.
    
    Args:
        vib_data: VibrationalData object
        molecule_name: Name for plot title
    """
    # Extract frequencies and intensities
    modes_with_ir = [m for m in vib_data.modes if m.ir_intensity is not None and not m.is_imaginary]
    
    if not modes_with_ir:
        print("No IR intensities available")
        return
    
    frequencies = [m.frequency for m in modes_with_ir]
    intensities = [m.ir_intensity for m in modes_with_ir]
    mode_numbers = [m.mode_number for m in modes_with_ir]
    
    # Create hover text
    hover_text = [
        f"Mode {num}<br>Frequency: {freq:.1f} cm⁻¹<br>IR Intensity: {ir:.1f} km/mol"
        for num, freq, ir in zip(mode_numbers, frequencies, intensities)
    ]
    
    # Create bar chart
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=frequencies,
        y=intensities,
        width=10,
        marker=dict(color='steelblue', opacity=0.7),
        hovertext=hover_text,
        hoverinfo='text',
        name='IR Peaks'
    ))
    
    fig.update_layout(
        title=dict(
            text=f'Interactive IR Spectrum - {molecule_name}<br><sub>Click on bars to see mode number</sub>',
            x=0.5,
            xanchor='center'
        ),
        xaxis_title='Frequency (cm⁻¹)',
        yaxis_title='IR Intensity (km/mol)',
        hovermode='closest',
        width=1000,
        height=500,
        template='plotly_white'
    )
    
    fig.show()

# Example usage
if vib_datasets:
    first_molecule = list(vib_datasets.keys())[0]
    create_interactive_ir_spectrum(vib_datasets[first_molecule], first_molecule)

## 5. Comparing Vibrational Modes Across Molecules

Compare characteristic frequencies across different molecules.

In [ ]:
def compare_molecules_frequencies(vib_datasets: Dict, freq_range: tuple = (3000, 3800)):
    """
    Compare vibrational frequencies in a specific range across molecules.
    
    Args:
        vib_datasets: Dictionary of molecule names to VibrationalData
        freq_range: Tuple of (min_freq, max_freq) to filter
    """
    fig, ax = plt.subplots(figsize=(12, 6))
    
    colors = plt.cm.Set3(np.linspace(0, 1, len(vib_datasets)))
    
    for (name, vib_data), color in zip(vib_datasets.items(), colors):
        # Filter modes in frequency range
        modes_in_range = [
            m for m in vib_data.modes 
            if freq_range[0] <= m.frequency <= freq_range[1] and not m.is_imaginary
        ]
        
        frequencies = [m.frequency for m in modes_in_range]
        intensities = [m.ir_intensity if m.ir_intensity else 50 for m in modes_in_range]
        
        # Plot as scatter
        ax.scatter(frequencies, [name] * len(frequencies), s=intensities, 
                  c=[color], alpha=0.6, label=name)
    
    ax.set_xlabel('Frequency (cm⁻¹)', fontsize=12)
    ax.set_ylabel('Molecule', fontsize=12)
    ax.set_title(f'Vibrational Mode Comparison ({freq_range[0]}-{freq_range[1]} cm⁻¹)', 
                fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    ax.set_xlim(freq_range[0] - 50, freq_range[1] + 50)
    
    plt.tight_layout()
    plt.show()

# Example: Compare O-H stretch region (3000-3800 cm⁻¹)
if len(vib_datasets) > 1:
    compare_molecules_frequencies(vib_datasets, freq_range=(3000, 3800))

## 6. Identifying Characteristic Functional Group Vibrations

Find modes in specific frequency ranges characteristic of functional groups.

In [ ]:
# Define characteristic frequency ranges for common functional groups
FUNCTIONAL_GROUP_RANGES = {
    'O-H stretch': (3200, 3700),
    'N-H stretch': (3300, 3500),
    'C-H stretch (alkane)': (2850, 3000),
    'C=O stretch (carbonyl)': (1650, 1750),
    'C=C stretch (alkene)': (1620, 1680),
    'C-O stretch': (1000, 1300),
    'N=O stretch (nitro)': (1500, 1600),
}

def identify_functional_groups(vib_data, molecule_name: str = "Molecule"):
    """
    Identify potential functional groups based on characteristic frequencies.
    
    Args:
        vib_data: VibrationalData object
        molecule_name: Name for display
    """
    print(f"\nFunctional group analysis for {molecule_name}:")
    print("=" * 70)
    
    for group_name, (min_freq, max_freq) in FUNCTIONAL_GROUP_RANGES.items():
        # Find modes in this range
        matching_modes = [
            m for m in vib_data.modes
            if min_freq <= m.frequency <= max_freq and not m.is_imaginary
        ]
        
        if matching_modes:
            print(f"\n{group_name} ({min_freq}-{max_freq} cm⁻¹):")
            for mode in matching_modes:
                ir_str = f"IR: {mode.ir_intensity:.1f}" if mode.ir_intensity else "IR: N/A"
                print(f"  Mode {mode.mode_number}: {mode.frequency:.1f} cm⁻¹ ({ir_str})")

# Example usage
if vib_datasets:
    first_molecule = list(vib_datasets.keys())[0]
    identify_functional_groups(vib_datasets[first_molecule], first_molecule)

## 7. Creating Publication-Quality Figures

Generate high-resolution figures suitable for publications.

In [ ]:
def create_publication_figure(vib_data, mode_number: int, smiles: str, 
                              output_path: str = None, dpi: int = 300):
    """
    Create a publication-quality figure with multiple panels.
    
    Args:
        vib_data: VibrationalData object
        mode_number: Mode to visualize
        smiles: SMILES string for molecule
        output_path: Optional path to save PNG
        dpi: Resolution for saved image
    """
    # Create 3D visualization with arrows
    fig = draw_3D_rep(smiles=smiles, mode="ball+stick")
    fig = add_vibrations_to_figure(
        fig=fig,
        vib_data=vib_data,
        mode_number=mode_number,
        display_type="arrows",
        amplitude=1.5,
        arrow_color="#E63946",  # Professional red
    )
    
    # Update layout for publication
    mode = vib_data.get_mode(mode_number)
    ir_str = f", IR: {mode.ir_intensity:.1f} km/mol" if mode.ir_intensity else ""
    
    fig.update_layout(
        title=dict(
            text=f"Vibrational Mode {mode_number}<br><sub>{mode.frequency:.1f} cm⁻¹{ir_str}</sub>",
            font=dict(size=18, family="Arial, sans-serif"),
            x=0.5,
            xanchor='center'
        ),
        scene=dict(
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=1.5)
            ),
            bgcolor='white'
        ),
        paper_bgcolor='white',
        width=800,
        height=800,
        font=dict(family="Arial, sans-serif")
    )
    
    # Save if output path provided
    if output_path:
        fig.write_image(output_path, width=800, height=800, scale=dpi/100)
        print(f"Figure saved to {output_path}")
    
    fig.show()

# Example usage
if vib_datasets:
    first_molecule = list(vib_datasets.keys())[0]
    vib_data = vib_datasets[first_molecule]
    
    # Get most IR-active mode
    modes_with_ir = [m for m in vib_data.modes if m.ir_intensity is not None]
    if modes_with_ir:
        most_active = max(modes_with_ir, key=lambda m: m.ir_intensity)
        create_publication_figure(
            vib_data=vib_data,
            mode_number=most_active.mode_number,
            smiles="O",  # Replace with actual SMILES
            output_path="vibration_mode.png",
            dpi=300
        )

## 8. Exporting Animations

Save vibration animations as HTML files for sharing.

In [ ]:
def export_vibration_animation(vib_data, mode_number: int, mol, 
                               output_path: str = "vibration_animation.html"):
    """
    Create and export a vibration animation as HTML.
    
    Args:
        vib_data: VibrationalData object
        mode_number: Mode to animate
        mol: RDKit Mol object
        output_path: Path to save HTML file
    """
    fig = create_vibration_animation(
        vib_data=vib_data,
        mode_number=mode_number,
        mol=mol,
        amplitude=0.5,
        n_frames=30,
        mode="ball+stick",
        resolution=32
    )
    
    # Save as standalone HTML
    fig.write_html(output_path, auto_open=False)
    print(f"Animation saved to {output_path}")
    print(f"File size: {Path(output_path).stat().st_size / 1024:.1f} KB")

# Example usage
from rdkit import Chem
from rdkit.Chem import AllChem

if vib_datasets:
    first_molecule = list(vib_datasets.keys())[0]
    vib_data = vib_datasets[first_molecule]
    
    # Create RDKit molecule
    mol = Chem.MolFromSmiles("O")  # Replace with actual SMILES
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol, randomSeed=42)
    
    export_vibration_animation(
        vib_data=vib_data,
        mode_number=1,
        mol=mol,
        output_path="mode_1_animation.html"
    )

## 9. Statistical Analysis of Vibrational Modes

In [ ]:
def analyze_mode_statistics(vib_data):
    """
    Compute statistical properties of vibrational modes.
    
    Args:
        vib_data: VibrationalData object
    """
    frequencies = [m.frequency for m in vib_data.modes if not m.is_imaginary]
    intensities = [m.ir_intensity for m in vib_data.modes if m.ir_intensity is not None]
    
    print("\nVibrational Mode Statistics:")
    print("=" * 60)
    print(f"Total modes: {len(vib_data.modes)}")
    print(f"Imaginary modes: {sum(1 for m in vib_data.modes if m.is_imaginary)}")
    
    if frequencies:
        print(f"\nFrequencies (cm⁻¹):")
        print(f"  Min: {np.min(frequencies):.1f}")
        print(f"  Max: {np.max(frequencies):.1f}")
        print(f"  Mean: {np.mean(frequencies):.1f}")
        print(f"  Median: {np.median(frequencies):.1f}")
        print(f"  Std Dev: {np.std(frequencies):.1f}")
    
    if intensities:
        print(f"\nIR Intensities (km/mol):")
        print(f"  Min: {np.min(intensities):.2f}")
        print(f"  Max: {np.max(intensities):.2f}")
        print(f"  Mean: {np.mean(intensities):.2f}")
        print(f"  Total: {np.sum(intensities):.2f}")
    
    # Displacement analysis
    all_displacements = []
    for mode in vib_data.modes:
        if not mode.is_imaginary:
            disps = vib_data.get_displacement_magnitudes(mode.mode_number)
            all_displacements.extend(disps)
    
    if all_displacements:
        print(f"\nDisplacement Vectors (Å):")
        print(f"  Max: {np.max(all_displacements):.4f}")
        print(f"  Mean: {np.mean(all_displacements):.4f}")

# Example usage
if vib_datasets:
    for name, vib_data in vib_datasets.items():
        print(f"\n{'='*60}")
        print(f"Analysis for: {name}")
        print(f"{'='*60}")
        analyze_mode_statistics(vib_data)

## Summary

In this advanced notebook, you learned:

✅ Batch processing multiple frequency calculations
✅ Creating vibrational mode summary tables with pandas
✅ Visualizing IR spectra (both matplotlib and Plotly)
✅ Comparing modes across different molecules
✅ Identifying characteristic functional group vibrations
✅ Creating publication-quality figures
✅ Exporting animations as HTML files
✅ Statistical analysis of vibrational properties

## Next Steps

- Explore mode combinations and linear combinations
- Implement Gaussian broadening for smoother IR spectra
- Compare calculated spectra with experimental data
- Analyze isotope effects on vibrational frequencies
- Create interactive dashboards with Dash or Panel